# Tomography Guide

Go from collected tiltseries to picked tomograms with this workflow! Uses [AreTomo3](https://github.com/czimaginginstitute/AreTomo3), [batch_pytom_aretomo3](https://github.com/Phaips/batch_pytom_aretomo3) (an intermediary step), [pytom-match-pick](https://sbc-utrecht.github.io/pytom-match-pick/), and [rln2imod](https://github.com/Phaips/rln2mod) (an intermediary step for imod visualization).

A spinoff of https://tomoguide.github.io/, customized for the Herzik Lab's compute cluster!

Made with joy by Daniel, with the guidance and great help of Brian and Sarah!

#### Outline
1. [Installation](#Installation)*
2. [AreTomo3](#Running-AreTomo3-(motion-correction,-alignment,-3D-reconstruction))**
3. [Pytom Template & Mask Creation](#Running-pytom-match-pick-(template-based-particle-picking))*
4. [batch_pytom_aretomo3](#Running-batch_pytom_aretomo3-(intermediary-to-run-pytom-on-AreTomo3-output))*
5. [Pytom Template Picking](#Running-pytom-match-pick:-template-matching-&-particle-extraction)**
6. [Visualization](#Visualizing-Picks)*
7. [Appendix](#Appendix)

<sub>\*Mostly manual work done by the user</sub><br/>
<sub>**Manual work in creating a submit script + several hour to full-day job run on SLURM</sub>

## Installation

### Setting up AreTomo3

#### Editing your ~/.tcshrc file

Run the following command:

```
printf '%s\n%s\n%s\n%s\n%s\n%s\n' 'set path = ($path /home/daji/AreTomo3-build)' 'setenv PREFIX /home/daji/local/libtiff-4.5' 'setenv PATH $PREFIX/bin:$PATH' 'setenv CPATH $PREFIX/include' 'setenv LIBRARY_PATH $PREFIX/lib' 'setenv LD_LIBRARY_PATH $PREFIX/lib:$LD_LIBRARY_PATH' >> ~/.tcshrc
```

To verify AreTomo3 is properly installed, you should be able to:

```
ssh gpu-11
module load cuda/11.2
AreTomo3 --version
```

And the last command will print out: 

```
version 2.1.10, built on May 08, 2025
```

### Setting up pytom-match-pick

Run the following commands (should also be done on a gpu node):

If you are not `ssh`'d into a gpu (any gpu is fine, doesn't have to be gpu-11) and don't have cuda/11.2 loaded yet:

```
ssh gpu-11
module load cuda/11.2
```

Then run:

```
module load conda/home
conda create -n pytom_tm python=3
conda activate pytom_tm
pip install pytom-match-pick
pip uninstall cupy
pip install cupy-cuda11x
```

Restart your terminal. The following command should work properly:

```
ssh gpu-11
module load conda/home
conda activate pytom_tm
module load cuda/11.2
pytom_create_template.py --help
```

and prints:

```
usage: pytom_create_template.py [-h] -i INPUT_MAP [-o OUTPUT_FILE]
                                [--input-voxel-size-angstrom INPUT_VOXEL_SIZE_ANGSTROM]
                                --output-voxel-size-angstrom OUTPUT_VOXEL_SIZE_ANGSTROM [--center]
                                [--low-pass LOW_PASS] [-b BOX_SIZE] [--invert] [-m] [--log LOG]

Generate template from MRC density. -- Marten Chaillet (@McHaillet)
```

# Running the Pipeline

## Running AreTomo3 (motion correction, alignment, 3D reconstruction)

**For convenience, the help page on AreTomo3 is in the appendix [here](#AreTomo3-Full-Options).**

### Creating the submit script (modify accordingly and then save & submit on hooke)

After edits are made to the submit script below, feel free to save the SLURM submit script to hooke and run it, e.g:

```
touch my_aretomo3_submit_script.sh
nano my_aretomo3_submit_script.sh [a live terminal editor will open, you can ctrl-v your copied submit script; and then to write to the file, ctrl-o, "y" and enter, ctrl-x (or just ctrl-x if you don't want to save your changes]
sbatch my_aretomo3_submit_script.sh
```

#### AreTomo3 submit script template: 

```
#!/bin/tcsh

#SBATCH -o aretomo3.out%j
#SBATCH -D ./
#SBATCH -J aretomo3
#SBATCH --partition=gpu-cpu
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --ntasks-per-node=1
#SBATCH --gres=gpu:2
#SBATCH --cpus-per-task=20
#SBATCH --mail-user=daji@ucsd.edu
#SBATCH --mem=128G
#SBATCH --time=00-12:00:00

cd $SLURM_SUBMIT_DIR

module purge
module load cuda/11.2

# Primary user-configured variables
set files      = "/home/bcook/detergent_cocktail/AldoTomo25x/micrographs/" # All frames with mdocs in this folder will be processed; can also specify only one Position_X tilt series.
set gain_ref   = "/data/titan2/herzik_20250512_181646/20250512_103503_EER_GainReference.gain" # give the gainref file!
set outdir     = "aretomo_output_Aldo25x_4" # output directory of tomograms & relevant files
set apix       = 1.89
set tilt_axis  = -174.33
set dark_tol   = 0.7        # tolerance for removing dark images, 0.7 is default
set vol_z      = 2048       # volume z height for reconstruction
set amp_con    = 0.07       # amplitude contrast
set fm_dose    = 0.137      # dose per frame (total dosage divided by # of frames)
set lowpass    = 15         # low pass filter
set frame_cnt  = 24         # number of frames
# set align_z  = 1400       # not really needed since AreTomo3 can estimate it automatically

mkdir -p "$outdir"

# May have to add / remove / modify more of the hard coded parameters here, see help page in appendix for further guidance
AreTomo3 \
    -InPrefix  "$files" \
    -Insuffix  ".mdoc" \
    -Gain      "$gain_ref" \
    -OutDir    "$outdir" \
    -FlipGain  1 \
    -Gpu       $CUDA_VISIBLE_DEVICES \
    -PixSize   $apix \
    -McBin     1 \
    -McPatch   1 1 \
    -FmInt     $frame_cnt \
    -FmDose    $fm_dose \
    -SplitSum  1 \
    -VolZ      $vol_z \
    -TiltAxis  $tilt_axis \
    -AtBin     4 \
    -OutXF     1 \
    -OutImod   1 \
    -Wbp       1 \
    -FlipVol   1 \
    -TiltCor   1 \
    -Patch     0 0 \
    -DarkTol   $dark_tol \
    -CorrCTF   1 $lowpass \
    -Kv        300 \
    -Cs        2.7 \  # spherical abberation
    -AmpContrast $amp_con

```

## Running pytom-match-pick (template-based particle picking)

### Running pytom-match-pick: template creation

Template creation can be done without a submit script, just make sure you've already run:

```
ssh gpu-11
module load cuda/11.2
module load conda/home
conda activate pytom_tm
# and cd to correct directory 
```

And then run something similar to the following:

```
pytom_create_template.py \
 -i templates/emd_2938.map \
 -o templates/80S.mrc \
 --input-voxel 1.1 \
 --output-voxel 13.79 \
 --center \
 --low-pass 30 \
 --invert \
 --mirror \
 --box-size 60
```

The `-i` and `-o` flags are your input and output mrc files. `--input-voxel` is the input map voxel size and `--output` is the tomogram voxel size. `--low-pass` is applied before downsampling the input volume **(in Angstroms)** but `--box-size` is relative to the output dimensions and **in voxels**. If CTF with defocus is already applied `--invert` is not needed.

### Running pytom-match-pick: mask creation

Mask creation (in the shape of a sphere) can be done without a submit script, just make sure you've already run:

```
ssh gpu-11
module load cuda/11.2
module load conda/home
conda activate pytom_tm
# and cd to correct directory
```

And then run something similar to the following:

```
pytom_create_mask.py \
  -b 32 \ # box size of output, in voxels (the size of the volume that will contain the mask)
  -o 80s_ribosome_2_mask_radius25_sigma0.5.mrc \ # output file
  --voxel-size 12 \ # voxel size
  -r 10 \ # radius of the sphere mask, in voxels 
  -s 0.5 # sigma of gaussian drop-off around the mask edges in number of pixels; 0.5-1.0 is usually sufficient for tomograms with 20A-10A voxel sizes
```

## Running batch_pytom_aretomo3 (intermediary to run pytom on AreTomo3 output)

For convenience, the help page on batch_pytom_aretomo3 is in the appendix [here](#batch_pytom_aretomo3-Full-Options). The official repo with a guide can be found here: https://github.com/Phaips/batch_pytom_aretomo3. 

### Note that some things are manually set in the pytom template submit script following this, so some values set in this batch_pytom_aretomo3.py may not be reflected and the corresponding parameter updates should be done in the following submit script.

Preparing AreTomo3 output to be PyTom template picking compatabile can be without a submit script, just make sure you've already run:

```
ssh gpu-11
module load cuda/11.2
module load conda/home
conda activate pytom_tm
# and cd to correct directory, likely the folder where your AreTomo3 folder output exists
```

#### batch_pytom_aretomo3 sample command:

```
python /home/daji/detergent_cocktail/batch_pytom_aretomo3.py \
  -i /home/snarehood/CryoET/CCET_April25/Aretomo3_1 \
  -t /home/snarehood/CryoET/CCET_April25/pytom_mask/80S_ribosome_2.mrc \
  -m /home/snarehood/CryoET/CCET_April25/pytom_mask/80s_ribosome_2_mask_radius25_sigma0.5.mrc \
  -g 0 1 \ # GPU devices, should not need to change
  --voxel-size-angstrom 12 \
  --dose 1.89 \
  --particle-diameter 270 \
  --dry-run \ # always include this, don't modify
  --amplitude-contrast 0.07 # should correspond with AreTomo3
```

### Running pytom-match-pick: template matching & particle extraction

**For convenience, the help page on PyTom template matching is in the appendix [here](#Pytom-Template-Matching-Full-Options).**

### Creating the submit script (modify accordingly and then save & submit on hooke)

After edits are made to the submit script below, feel free to save the SLURM submit script to hooke and run it, e.g:

```
touch my_pytom_submit_script.sh
nano my_pytom_submit_script.sh [a live terminal editor will open, you can ctrl-v your copied submit script; and then to write to the file, ctrl-o, "y" and enter, ctrl-x (or just ctrl-x if you don't want to save your changes]
sbatch my_pytom_submit_script.sh
```

#### pytom-match-pick submit script template: 

```
#!/bin/bash -l
#SBATCH -o pytom.out%j
#SBATCH -D ./
#SBATCH -J pytom_2
#SBATCH --partition=gpu-cpu
#SBATCH --ntasks=1
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=8
#SBATCH --gres=gpu:2
#SBATCH --mail-user=daji@ucsd.edu
#SBATCH --mem=128G
#SBATCH --time=05-00:00:00


# === START User-Modified Configuration Variables ===
# Full path to the input volume MRC file
INPUT_VOLUME_FULL_PATH="/home/bcook/detergent_cocktail/AldoTomo25x/aretomo_output_Aldo25x_3/Position_2_2_Vol.mrc"
# Dose accumulation file
DOSE_ACCUMULATION="/home/bcook/detergent_cocktail/AldoTomo25x/aretomo_output_Aldo25x_3_picking/Position_2_2/Position_2_2_exposure.txt"
# Defocus
DEFOCUS="/home/bcook/detergent_cocktail/AldoTomo25x/aretomo_output_Aldo25x_3_picking/Position_2_2/Position_2_2_defocus.txt"
# Tilt Angles
TILT_ANGLES="/home/bcook/detergent_cocktail/AldoTomo25x/aretomo_output_Aldo25x_3_picking/Position_2_2/Position_2_2.tlt"
# Template volume mrc file
TEMPLATE="/home/bcook/detergent_cocktail/AldoTomo1x/partPick/J761_tomoMap2.mrc"
# Template mask mrc file
TEMPLATE_MASK="/home/bcook/detergent_cocktail/AldoTomo1x/partPick/NewMask_bx40r12.mrc"
# Directory where the output and intermediate files will be saved
OUTPUT_DIR="/home/daji/detergent_cocktail/AldoTomo25x/submission/Position_2_2/"
# Split values for pytom_match_template.py, usually don't need to change, but can increase if GPU runs out of memory
SPLIT="4 4 2"

# Particle diameter for pytom_match_template.py
PARTICLE_DIAMETER="110.0"
# High-pass filter for pytom_match_template.py
HIGH_PASS="400"
# Voxel size in Angstroms for pytom_match_template.py
VOXEL_SIZE_ANGSTROM="7.56"
# Amplitude contrast for pytom_match_template.py
AMPLITUDE_CONTRAST="0.07"
# Spherical aberration for pytom_match_template.py
SPHERICAL_ABERRATION="2.7"
# Voltage for pytom_match_template.py
VOLTAGE="300"

# Particle count for pytom_extract_candidates.py
PARTICLE_COUNT="10000"
# Threshold for pytom_extract_candidates.py -c option
THRESHOLD="0.3"
# === END User-Modified Configuration Variables ===

# Derived config variables (shouldn't have to modify anything here and below)
INPUT_VOLUME_BASENAME=$(basename "${INPUT_VOLUME_FULL_PATH}")
BASENAME_NO_EXT="${INPUT_VOLUME_BASENAME%.mrc}"
JOB_JSON_PATH="${OUTPUT_DIR}/${BASENAME_NO_EXT}_job.json"
PARTICLES_STAR_FILE_BASE="${OUTPUT_DIR}/${BASENAME_NO_EXT}_particles"

mkdir -p "${OUTPUT_DIR}"

module purge
module load cuda/11.2
module load conda/home
conda activate pytom_tm

echo "Splitting tomogram by ${SPLIT}"

pytom_match_template.py \
  -v "${INPUT_VOLUME_FULL_PATH}" \
  --dose-accumulation "${DOSE_ACCUMULATION}" \
  --defocus "${DEFOCUS}" \
  -a "${TILT_ANGLES}" \
  -t "${TEMPLATE}" \
  -d "${OUTPUT_DIR}" \
  -m "${TEMPLATE_MASK}" \
  --particle-diameter "${PARTICLE_DIAMETER}" \
  -s ${SPLIT} \
  --high-pass "${HIGH_PASS}" \
  --voxel-size-angstrom "${VOXEL_SIZE_ANGSTROM}" \
  -g 0 1 \
  --per-tilt-weighting \
  --amplitude-contrast "${AMPLITUDE_CONTRAST}" \
  --spherical-aberration "${SPHERICAL_ABERRATION}" \
  --voltage "${VOLTAGE}"

pytom_extract_candidates.py \
  -j "${JOB_JSON_PATH}" \
  -n "${PARTICLE_COUNT}" \
  -c "${THRESHOLD}" \
  --relion5-compat

module unload cuda
module load imod

cp "${PARTICLES_STAR_FILE_BASE}.star" "${PARTICLES_STAR_FILE_BASE}_relion_compat.star"

cd $OUTPUT_DIR

read x y z <<< "$(grep -A3 '"tomo_shape"' ${JOB_JSON_PATH} | grep -oE '[0-9]+' | head -3 | xargs)"
python /home/daji/rln2mod/rln2mod.py --x $x --y $y --z $z

module purge
module load cuda/11.2
module load conda/home
conda activate pytom_tm

python /home/daji/detergent_cocktail/RlnToArtiaX.py --input "${PARTICLES_STAR_FILE_BASE}_relion_compat.star" --output "${PARTICLES_STAR_FILE_BASE}_ArtiaX.star" --job-json "${JOB_JSON_PATH}"
```



## Visualizing Picks

### Visualizing in IMOD (https://bio3d.colorado.edu/imod/download.html)

#### NOTE: Documentation can be improved here.

For Mac, when it comes to download IMOD (if not already installed), usually the command-line self-installing file is more reliable (from personal experience). After downloading the corresponding *.sh file, you would simply just run `bash imod_xxxx.sh` to install it.

After running the submit script from above, you should have a `mod` folder in your specified output directory with a `.mod` file that you can load into IMOD to visualize.

### Visualizing in ArtiaX (https://github.com/FrangakisLab/ArtiaX)

ChimeraX >= 1.3 is needed to install ArtiaX. Installation instructions can be found here: https://github.com/FrangakisLab/ArtiaX?tab=readme-ov-file#using-the-chimerax-toolshed

After running the submit script from above, you should have a `*_ArtiaX.star` file in your specified output directory that you can load into ArtiaX with your tomogram.

## Appendix


### AreTomo3 Full Options
```
Full AreTomo3 options:

Usage: AreTomo3 Tags
     ******  Common Parameters  *****
-InPrefix      
  1. Prefix of input file name(s), ogether with Insuffix
     and InSkips, is used to form either a single or subset
     for file name(s), which are processed by AreTomo3.
  2. If the suffix is mdoc, any mdoc file that starts with
     the prefix string will be selected.
  3. If the suffix is mrc, any mrc file that starts with
     the prefix string will be selected and processed.
  4. The prefix can also be the path of a folder containing
     the movie files (tiff or eer) or tilt series (mrc).
  5. Note that movie files must be in the same directory
     as the mdoc files.

               
  1. If MDOC files have .mdoc file name extension, then
     .mdoc should be given after 0I\. If another extension
     is used, it should be used instead.

  1. If a MDOC file contains any string given behind %s,
     those MDOC files will not be processed.


-OutDir        
  1. Path to output folder to store generated tilt series, tomograms,
     and alignment files.

-PixSize       
  1. Pixel size in A of input stack in angstrom.

-kV            
  1. High tension in kV needed for dose weighting.
  2. Default is 300.

-Cs            
  1. Spherical aberration in mm for CTF estimation.

-FmDose        
  1. Per frame dose in e/A2.

-SplitSum      
   1. Generate odd and even sums using odd and even frames.
   2. The default value is 1, which enables the split, 0
      disables this function.
   3. When enabled, 3 tilt series and 3 tomograms will be
      generated. Tilt series and tomograms generated from
      odd and even sums are appended _ODD and _EVN in
      their file names, respectively.
   4. When disabled, odd and even tilt series and tomograms
      will not be generated. Tilt series and tomogram from
      full sums are generated only.

-Cmd           
  1. Default 0 starts processing from motion correction.
  2. -Cmd 1 starts processing from tilt series alignment
     including CTF estimation, correction, tomographic
     alignment and reconstruction.
  3. -Cmd 2 starts processing from CTF correction and
     then tomographic reconstruction.
  4. -Cmd 1 and -Cmd 2 ignore -Resume.

-Resume        
  1. Default 0 processes all the data.
  2. -Resume 1 starts from what are left by skipping all the mdoc
     files in MdocDone.txt file in the output folder.

-Gpu           
   GPU IDs. Default 0.
   For multiple GPUs, separate IDs by space.
   For example, -Gpu 0 1 2 3 specifies 4 GPUs.

-DefectFile    
1. Defect file stores entries of defects on camera.
2. Each entry corresponds to a rectangular region in image.
   The pixels in such a region are replaced by neighboring
   good pixel values.
3. Each entry contains 4 integers x, y, w, h representing
   the x, y coordinates, width, and heights, respectively.

-Gain          
1. MRC or TIFF file that stores the gain reference.
2. Falcon camera produced .gain file can also be used
   since it is a TIFF file.

-Dark          
  1. MRC file that stores the dark reference. If not
     specified, dark subtraction will be skipped.
  2. If -RotGain and/or -FlipGain is specified, the
     dark reference will also be rotated and/or flipped.

-McPatch       
  1. It is followed by numbers of patches in x and y dimensions.
  2. The default values are 1 1, meaning only full-frame
     based alignment is performed.

-McIter        
   Maximum iterations for iterative alignment,
   default 7 iterations.

-McTol         
   Tolerance for iterative alignment,
   default 0.5 pixel.

-McBin         
   Binning performed in Fourier space, default 1.0.

-Group         
   1. Group every specified number of frames by adding
      them together. The alignment is then performed
      on the group sums. The so measured motion is
      interpolated to each raw frame.
   2. The 1st integer is for gobal alignment and the
      2nd is for patch alignment.

-FmRef         
   Specify a frame in the input movie stack to be the
   reference to which all other frames are aligned. The
   reference is 1-based index in the input movie stack
   regardless how many frames will be thrown. By default
   the reference is set to be the central frame.

-RotGain       
   Rotate gain reference counter-clockwise.
   0 - no rotation, default,
   1 - rotate 90 degree,
   2 - rotate 180 degree,
   3 - rotate 270 degree.

-FlipGain      
   Flip gain reference after gain rotation.
   0 - no flipping, default,
   1 - flip upside down, 
   2 - flip left right.

-InvGain       
   Inverse gain value at each pixel (1/f). If a orginal
   value is zero, the inversed value is set zero.
   This option can be used together with flip and
   rotate gain reference.

-Mag           
   1. Correct anisotropic magnification by stretching
      image along the major axis, the axis where the
      lower magificantion is detected.
   2. Three inputs are needed including magnifications
      along major and minor axes and the angle of the
      major axis relative to the image x-axis in degree.
   3. By default no correction is performed.

-InFmMotion    
   1. 1 - Account for in-frame motion.
      0 - Do not account for in-frame motion.

-TiltAxis 
   1. User provided angle of tilt axis in degree. If users
      do not provide one, AreTomo3 will search in full range.
   2. If users provide one and do not want AreTomo3 to refine
      it, add -1 after the provided tilt axis.
   3. Otherwise, AreTomo3 regines the provided value in a
      in a smaller range.

-AlignZ   
   Volume height for alignment, default 256

-VolZ     
   1. Volume z height for reconstrunction. It must be
      greater than 0 to reconstruct a volume.
   2. Default is 0, only aligned tilt series will
      generated.

-ExtZ     
   1. Extra volume z height for reconstrunction. This is
      the  space added to the estimated sample thickness for
      the final reconstruction of tomograms.
   2. This setting is relevant only when -VolZ -1 is set,
      which means users want to use the estimated sample
      thickness.

      greater than 0 to reconstruct a volume.
   2. Default is 0, only aligned tilt series will
      generated.

-AtBin    
   1. Binnings for tomograms with respect to motion
      corrected tilt series. Users can specify two floats.
       2. The first number is required and 1.0 is default.
       3. The second number is optional. By default, the
      second resolution is disabled. It is actived only
      when users provide a different number.

-TiltCor  
   1. Correct the offset of tilt angle.
   2. This argument can be followed by two values. The
      first value can be -1, 0, or 1. and the  default is 0,
      indicating the tilt offset is measured for alignment
      only  When the value is 1, the offset is applied to
      reconstion too. When a negative value is given, tilt
      is not measured not applied.
   3. The second value is user provided tilt offset. When it
      is given, the measurement is disabled.

-ReconRange
   1. It specifies the min and max tilt angles from which
      a 3D volume will be reconstructed. Any tilt image
      whose tilt ange is outside this range is exclueded
      in the reconstruction.

-AmpContrast
   1. Amplitude contrast, default 0.07

-ExtPhase 
   1. Guess of phase shift and search range in degree.
   2. Only required for CTF estimation and with
   3. Phase plate installed.

-FlipVol  
   1. By giving a non-zero value, the reconstructed
      volume is saved in xyz fashion. The default is
      xzy.
-FlipInt  
  1. Flip the intensity of the volume to make structure white.
     Default 0 means no flipping. Non-zero value flips.
-Sart     
   1. Specify number of SART iterations and number
      of projections per update. The default values
      are 15 and 5, respectively

-Wbp      
   1. By specifying 1, weighted back projection is enabled
      to reconstruct volume.

-DarkTol  
   1. Set tolerance for removing dark images. The range is
      in (0, 1). The default value is 0.7. The higher value is
      more restrictive.

-OutXF    
   1. When set by giving no-zero value, IMOD compatible
      XF file will be generated.

-OutImod  
   1. It generates the Imod files needed by Relion4 or Warp
      for subtomogram averaging. These files are saved in the
      subfolder named after the output MRC file name.
   2. 0: default, do not generate any IMod files.
   3. 1: generate IMod files needed for Relion 4.
   4. 2: generate IMod files needed for WARP.
   5. 3: generate IMod files when the aligned tilt series
         is used as the input for Relion 4 or WARP.

-Align    
   1. Skip alignment when followed by 0. This option is
      used when the input MRC file is an aligned tilt series.
      The default value is 1.

-IntpCor  
   1. When enabled, the correction for information loss due
      to linear interpolation will be perform. The default
      setting value 1 enables the correction.

-CorrCTF  
   1. When enabled, local CTF correction is performed on
      raw tilt series. By default this function is enabled.
   2. Passing 0 disables this function.
```


### batch_pytom_aretomo3 Full Options

```
usage: batch_pytom_aretomo3.py [-h] -i ARETOMO_DIR [-d OUTPUT_DIR] [--bmask-dir BMASK_DIR] -t TEMPLATE -m MASK -g GPU_IDS [GPU_IDS ...]
                               --voxel-size-angstrom VOXEL_SIZE_ANGSTROM --dose DOSE [--include INCLUDE [INCLUDE ...]] [--exclude EXCLUDE [EXCLUDE ...]]
                               [--dry-run] (--particle-diameter PARTICLE_DIAMETER | --angular-search ANGULAR_SEARCH) [--non-spherical-mask]
                               [--z-axis-rotational-symmetry Z_AXIS_ROTATIONAL_SYMMETRY] [-s X Y Z] [--search-x START END] [--search-y START END]
                               [--search-z START END] [--tomogram-ctf-model {phase-flip}] [-r] [--half-precision] [--rng-seed RNG_SEED]
                               [--per-tilt-weighting] [--low-pass LOW_PASS] [--high-pass HIGH_PASS] [--amplitude-contrast AMPLITUDE_CONTRAST]
                               [--spherical-aberration SPHERICAL_ABERRATION] [--voltage VOLTAGE] [--phase-shift PHASE_SHIFT]
                               [--defocus-handedness {-1,0,1}] [--spectral-whitening] [--relion5-tomograms-star RELION5_TOMOGRAMS_STAR]
                               [--log {info,debug}] [--partition PARTITION] [--ntasks NTASKS] [--nodes NODES] [--ntasks-per-node NTASKS_PER_NODE]
                               [--cpus-per-task CPUS_PER_TASK] [--gres GRES] [--mem MEM] [--qos QOS] [--time TIME] [--mail-type MAIL_TYPE]

Batch-submit PyTom template matching for AreTomo3 tomograms

options:
  -h, --help            show this help message and exit
  -i ARETOMO_DIR, --aretomo-dir ARETOMO_DIR
                        AreTomo3 output directory
  -d OUTPUT_DIR, --output-dir OUTPUT_DIR
                        Top-level folder for per-tomogram subfolders
  --bmask-dir BMASK_DIR
                        Directory containing binary mask files (format has to be: <prefix>.mrc !!)
  -t TEMPLATE, --template TEMPLATE
                        Template MRC file
  -m MASK, --mask MASK  Mask MRC file
  -g GPU_IDS [GPU_IDS ...], --gpu-ids GPU_IDS [GPU_IDS ...]
                        GPU IDs for pytom_match_template.py (e.g. -g 0)
  --voxel-size-angstrom VOXEL_SIZE_ANGSTROM
                        Voxel size in Å
  --dose DOSE           Electron dose per tilt (e-/Å²)
  --include INCLUDE [INCLUDE ...]
                        Process only these prefixes
  --exclude EXCLUDE [EXCLUDE ...]
                        Exclude these prefixes
  --dry-run             Generate scripts only; don't sbatch
  --particle-diameter PARTICLE_DIAMETER
                        Particle diameter in Å (Crowther sampling)
  --angular-search ANGULAR_SEARCH
                        Override angular search
  --non-spherical-mask  Enable non-spherical mask
  --z-axis-rotational-symmetry Z_AXIS_ROTATIONAL_SYMMETRY
                        Z-axis rotational symmetry
  -s X Y Z, --volume-split X Y Z
                        Split volume into blocks X Y Z
  --search-x START END  Search range along x
  --search-y START END  Search range along y
  --search-z START END  Search range along z
  --tomogram-ctf-model {phase-flip}
                        CTF model used in reconstruction
  -r, --random-phase-correction
                        Enable random-phase correction
  --half-precision      Use float16 output
  --rng-seed RNG_SEED   RNG seed (default: 69)
  --per-tilt-weighting  Enable per-tilt weighting
  --low-pass LOW_PASS   Low-pass filter Å
  --high-pass HIGH_PASS
                        High-pass filter Å
  --amplitude-contrast AMPLITUDE_CONTRAST
                        Amplitude contrast (default: 0.07)
  --spherical-aberration SPHERICAL_ABERRATION
                        Spherical aberration mm (default: 2.7)
  --voltage VOLTAGE     Voltage kV (default: 300)
  --phase-shift PHASE_SHIFT
                        Phase shift degrees
  --defocus-handedness {-1,0,1}
                        Defocus handedness (only if set)
  --spectral-whitening  Enable spectral whitening
  --relion5-tomograms-star RELION5_TOMOGRAMS_STAR
                        RELION5 tomograms.star path (overrides tilts/CTF files)
  --log {info,debug}    Logging level
  --partition PARTITION
                        SLURM partition
  --ntasks NTASKS       SLURM ntasks
  --nodes NODES         SLURM nodes
  --ntasks-per-node NTASKS_PER_NODE
                        SLURM tasks/node
  --cpus-per-task CPUS_PER_TASK
                        SLURM CPUs/task
  --gres GRES           SLURM gres
  --mem MEM             SLURM mem (GB)
  --qos QOS             SLURM QoS
  --time TIME           SLURM time
  --mail-type MAIL_TYPE
                        SLURM mail-type
```

### Pytom Template Matching Full Options

```
usage: pytom_match_template.py [-h] -t TEMPLATE -v TOMOGRAM [-d DESTINATION] -m MASK [--non-spherical-mask]
                               [--particle-diameter PARTICLE_DIAMETER] [--angular-search ANGULAR_SEARCH]
                               [--z-axis-rotational-symmetry Z_AXIS_ROTATIONAL_SYMMETRY]
                               [-s VOLUME_SPLIT VOLUME_SPLIT VOLUME_SPLIT] [--search-x SEARCH_X SEARCH_X]
                               [--search-y SEARCH_Y SEARCH_Y] [--search-z SEARCH_Z SEARCH_Z]
                               [--tomogram-mask TOMOGRAM_MASK] [-a TILT_ANGLES [TILT_ANGLES ...]]
                               [--per-tilt-weighting] [--voxel-size-angstrom VOXEL_SIZE_ANGSTROM]
                               [--low-pass LOW_PASS] [--high-pass HIGH_PASS] [--dose-accumulation DOSE_ACCUMULATION]
                               [--defocus DEFOCUS] [--amplitude-contrast AMPLITUDE_CONTRAST]
                               [--spherical-aberration SPHERICAL_ABERRATION] [--voltage VOLTAGE]
                               [--phase-shift PHASE_SHIFT] [--tomogram-ctf-model {phase-flip}]
                               [--defocus-handedness {-1,0,1}] [--spectral-whitening] [-r] [--half-precision]
                               [--rng-seed RNG_SEED] [--relion5-tomograms-star RELION5_TOMOGRAMS_STAR]
                               -g GPU_IDS [GPU_IDS ...] [--log LOG]

Run template matching. -- Marten Chaillet (@McHaillet)

options:
  -h, --help            show this help message and exit

Template, search volume, and output:
  -t, --template TEMPLATE
                        Template; MRC file. Object should match the contrast of the tomogram: if the tomogram has
                        black ribosomes, the reference should be black. (pytom_create_template.py has an option to
                        invert contrast)
  -v, --tomogram TOMOGRAM
                        Tomographic volume; MRC file.
  -d, --destination DESTINATION
                        Folder to store the files produced by template matching.

Mask:
  -m, --mask MASK       Mask with same box size as template; MRC file.
  --non-spherical-mask  Flag to set when the mask is not spherical. It adds the required computations for non-
                        spherical masks and roughly doubles computation time.

Angular search:
  --particle-diameter PARTICLE_DIAMETER
                        Provide a particle diameter (in Angstrom) to automatically determine the angular sampling
                        using the Crowther criterion. For the max resolution, (2 * pixel size) is used unless a low-
                        pass filter is specified, in which case the low-pass resolution is used. For non-globular
                        macromolecules choose the diameter along the longest axis.
  --angular-search ANGULAR_SEARCH
                        This option overrides the angular search calculation from the particle diameter. If given a
                        float it will generate an angle list with healpix for Z1 and X1 and linear search for Z2. The
                        provided angle will be used as the maximum for the linear search and for the mean angle
                        difference from healpix.Alternatively, a .txt file can be provided with three Euler angles (in
                        radians) per line that define the angular search. Angle format is ZXZ anti-clockwise (see:
                        https://www.ccpem.ac.uk/user_help/rotation_conventions.php).
  --z-axis-rotational-symmetry Z_AXIS_ROTATIONAL_SYMMETRY
                        Integer value indicating the rotational symmetry of the template around the z-axis. The length
                        of the rotation search will be shortened through division by this value. Only works for
                        template symmetry around the z-axis.

Volume control:
  -s, --volume-split VOLUME_SPLIT VOLUME_SPLIT VOLUME_SPLIT
                        Split the volume into smaller parts for the search, can be relevant if the volume does not fit
                        into GPU memory. Format is x y z, e.g. --volume-split 1 2 1
  --search-x SEARCH_X SEARCH_X
                        Start and end indices of the search along the x-axis, e.g. --search-x 10 490
  --search-y SEARCH_Y SEARCH_Y
                        Start and end indices of the search along the y-axis, e.g. --search-x 10 490
  --search-z SEARCH_Z SEARCH_Z
                        Start and end indices of the search along the z-axis, e.g. --search-x 30 230
  --tomogram-mask TOMOGRAM_MASK
                        Here you can provide a mask for matching with dimensions (in pixels) equal to the tomogram. If
                        a subvolume only has values <= 0 for this mask it will be skipped.

Filter control:
  -a, --tilt-angles TILT_ANGLES [TILT_ANGLES ...]
                        Tilt angles of the tilt-series, either the minimum and maximum values of the tilts (e.g.
                        --tilt-angles -59.1 60.1) or a .rawtlt/.tlt file with all the angles (e.g. --tilt-angles
                        tomo101.rawtlt). In case all the tilt angles are provided a more elaborate Fourier space
                        constraint can be used
  --per-tilt-weighting  Flag to activate per-tilt-weighting, only makes sense if a file with all tilt angles have been
                        provided. In case not set, while a tilt angle file is provided, the minimum and maximum tilt
                        angle are used to create a binary wedge. The base functionality creates a fanned wedge where
                        each tilt is weighted by cos(tilt_angle). If dose accumulation and CTF parameters are provided
                        these will all be incorporated in the tilt-weighting.
  --voxel-size-angstrom VOXEL_SIZE_ANGSTROM
                        Voxel spacing of tomogram/template in angstrom, if not provided will try to read from the MRC
                        files. Argument is important for band-pass filtering!
  --low-pass LOW_PASS   Apply a low-pass filter to the tomogram and template. Generally desired if the template was
                        already filtered to a certain resolution. Value is the resolution in A.
  --high-pass HIGH_PASS
                        Apply a high-pass filter to the tomogram and template to reduce correlation with large low
                        frequency variations. Value is a resolution in A, e.g. 500 could be appropriate as the CTF is
                        often incorrectly modelled up to 50nm.
  --dose-accumulation DOSE_ACCUMULATION
                        Here you can provide a file that contains the accumulated dose at each tilt angle, assuming
                        the same ordering of tilts as the tilt angle file. Format should be a .txt file with on each
                        line a dose value in e-/A2.
  --defocus DEFOCUS     Here you can provide an IMOD defocus (.defocus) file (version 2 or 3) , a text (.txt) file
                        with a single defocus value per line (in μm), or a single defocus value (in μm). The value(s),
                        together with the other ctf parameters (amplitude contrast, voltage, spherical abberation),
                        will be used to create a 3D CTF weighting function. IMPORTANT: if you provide this, the input
                        template should not be modulated with a CTF beforehand. If it is a reconstruction it should
                        ideally be Wiener filtered.
  --amplitude-contrast AMPLITUDE_CONTRAST
                        Amplitude contrast fraction for CTF.
  --spherical-aberration SPHERICAL_ABERRATION
                        Spherical aberration for CTF in mm.
  --voltage VOLTAGE     Voltage for CTF in keV.
  --phase-shift PHASE_SHIFT
                        Phase shift (in degrees) for the CTF to model phase plates.
  --tomogram-ctf-model {phase-flip}
                        Optionally, you can specify if and how the CTF was corrected during reconstruction of the
                        input tomogram. This allows match-pick to match the weighting of the template to the tomogram.
                        Not using this option is appropriate if the CTF was left uncorrected in the tomogram. Option
                        'phase-flip' : appropriate for IMOD's strip-based phase flipping or reconstructions generated
                        with novaCTF/3dctf.
  --defocus-handedness {-1,0,1}
                        Specify the defocus handedness for defocus gradient correction of the CTF in each subvolumes.
                        The more subvolumes in x and z, the finer the defocus gradient will be corrected, at the cost
                        of increased computing time. It will only have effect for very clean and high-resolution data,
                        such as isolated macromolecules. IMPORTANT: only works in combination with --volume-split ! A
                        value of 0 means no defocus gradient correction (default), 1 means correction assuming correct
                        handedness (as specified in Pyle and Zianetti (2021)), -1 means the handedness will be
                        inverted. If uncertain better to leave off as an inverted correction might hamper results.
  --spectral-whitening  Calculate a whitening filtering from the power spectrum of the tomogram; apply it to the
                        tomogram patch and template. Effectively puts more weight on high resolution features and
                        sharpens the correlation peaks.

Additional options:
  -r, --random-phase-correction
                        Run template matching simultaneously with a phase randomized version of the template, and
                        subtract this 'noise' map from the final score map. For this method please see STOPGAP as a
                        reference: https://doi.org/10.1107/S205979832400295X .
  --half-precision      Return and save all output in float16 instead of the default float32
  --rng-seed RNG_SEED   Specify a seed for the random number generator used for phase randomization for consistent
                        results!
  --relion5-tomograms-star RELION5_TOMOGRAMS_STAR
                        Here, you can provide a path to a RELION5 tomograms.star file (for example from a tomogram
                        reconstruction job). pytom-match-pick will fetch all the tilt-series metadata from this file
                        and overwrite all other metadata options.

Device control:
  -g, --gpu-ids GPU_IDS [GPU_IDS ...]
                        GPU indices to run the program on.
```